## Advanced options to customize Counterfactual Explanations

Here we discuss a few ways to change DiCE's behavior. 

* Train a custom ML model 
* Changing feature weights that decide relative importance of features in perturbation
* Trading off between proximity and diversity goals
* Selecting the features to change

In [1]:
# import DiCE
import dice_ml
from dice_ml.utils import helpers # helper functions

# Tensorflow libraries
import tensorflow as tf
from tensorflow import keras

### Loading dataset

We use "adult" income dataset from UCI Machine Learning Repository (https://archive.ics.uci.edu/ml/datasets/adult). For demonstration purposes, we transform the data as detailed in **dice_ml.utils.helpers** module. 

In [2]:
dataset = helpers.load_adult_income_dataset()

In [3]:
dataset.head()

,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,39,Government,Bachelors,Single,White-Collar,White,Male,40,0
1,50,Self-Employed,Bachelors,Married,White-Collar,White,Male,13,0
2,38,Private,HS-grad,Divorced,Blue-Collar,White,Male,40,0
3,53,Private,School,Married,Blue-Collar,Other,Male,40,0
4,28,Private,Bachelors,Married,Professional,Other,Female,40,0


In [4]:
d = dice_ml.Data(dataframe=dataset, continuous_features=['age', 'hours_per_week'], outcome_name='income')

### 1. Training a custom ML model

Below, we build an Artificial Neural Network based on Keras Tensorflow framework.

In [5]:
# seeding random numbers for reproducability
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

In [6]:
sess = tf.InteractiveSession()

train, _ = d.split_data(d.normalize_data(d.one_hot_encoded_data))
X_train = train.loc[:, train.columns != 'income']
y_train = train.loc[:, train.columns == 'income']

ann_model = keras.Sequential()
ann_model.add(keras.layers.Dense(20, input_shape=(X_train.shape[1],), kernel_regularizer=keras.regularizers.l2(0.01), activation=tf.nn.relu))
ann_model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

ann_model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.01), metrics=['accuracy'])
ann_model.fit(X_train, y_train, validation_split=0.20, epochs=50, verbose=0)


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [7]:
m = dice_ml.Model(model=ann_model) 

### Generate diverse counterfactuals

In [8]:
# initiate DiCE
exp = dice_ml.Dice(d, m)

In [9]:
# query instance in the form of a dictionary; keys: feature name, values: feature value
query_instance = {'age':22, 
                  'workclass':'Private', 
                  'education':'HS-grad', 
                  'marital_status':'Single', 
                  'occupation':'Service',
                  'race': 'White', 
                  'gender':'Female', 
                  'hours_per_week': 45}

In [10]:
# generate counterfactuals
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=4, desired_class="opposite")

Diverse Counterfactuals found! total time taken: 00 min 10 sec


In [11]:
# visualize the resutls
dice_exp.visualize_as_dataframe()

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,22.0,Private,HS-grad,Single,Service,White,Female,45.0,0.009978



Diverse Counterfactual set (new outcome : 1)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,90.0,Private,Prof-school,Married,Service,Other,Male,89.0,0.823
1,89.0,Private,Masters,Single,White-Collar,White,Male,99.0,0.823
2,90.0,Self-Employed,HS-grad,Married,Professional,White,Female,99.0,0.808
3,90.0,Government,Doctorate,Married,Service,White,Female,99.0,0.823


We transform continuous features and one-hot-encode categorical features to fall between 0 and 1 in order to handle relative scale of features. However, this also means that the relative ease of changing continuous features is higher than categorical features, when the total number of continuous features are very less compared to the total number of categories of all categorical variables combined. This is reflected in the above table where continuous features (*age* and *hours_per_week*) have been varied to reach their maximum values (*max_age:* 90, *max_hours_per_week:* 99) for all four counterfactuals. One way to handle this issue is to modify the weights for continuous features that reflect the relative difficulty in changing them (see our [paper](https://arxiv.org/pdf/1905.07697.pdf) for more details).

### 2. Changing feature weights

Median Absolute Deviation (MAD) of a continuous feature tells the variability of the feature, and is more robust than standard deviation as is less affected by outliers and non-normality. The inverse of MAD would then imply the ease of varying the feature and hence can be used as feature weights in our optimization to reflect the difficulty of changing a continuous feature. DiCE allows this through a *feature weight* parameter.

In [12]:
# get MAD
mads = d.get_mads_from_training_data(normalized=True)

# create feature weights
feature_weights = {}
for feature in mads:
    feature_weights[feature] = round(1/mads[feature], 2)
print(feature_weights)

{'age': 7.3, 'hours_per_week': 24.5}


The above feature weights imply that changing *age* is approximately seven times more difficult than changing categorical variables, and changing *hours_per_week* is approximately three times more difficult than chaning *age*.

In [13]:
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=4, desired_class="opposite", 
                                        feature_weights=feature_weights)

Diverse Counterfactuals found! total time taken: 00 min 08 sec


In [14]:
dice_exp.visualize_as_dataframe()

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,22.0,Private,HS-grad,Single,Service,White,Female,45.0,0.009978



Diverse Counterfactual set (new outcome : 1)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,18.0,Private,Doctorate,Married,Professional,White,Male,47.0,0.802
1,90.0,Private,Masters,Married,Service,White,Female,23.0,0.699
2,17.0,Private,Doctorate,Married,White-Collar,White,Female,70.0,0.745
3,39.0,Private,Masters,Married,White-Collar,White,Male,61.0,0.823


As we see from above table, the input of feature weights to our optimization has resulted in much more feasible counterfactuals than before. 

### 3. Trading off between proximity and diversity goals

We acknowledge that not all counterfactual explanations may be feasible for a user. In general, counterfactuals closer to an individual's profile will be more feasible. Diversity is also important to help an individual choose between multiple possible options. DiCE allows tunable parameters *proximity_weight* (default: 0.5) and *diversity_weight* (default: 1.0) to handle proximity and diversity respectively. Below, we increse the proximity weight and see how the counterfactuals change.

In [15]:
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=4, desired_class="opposite", 
                                        feature_weights=feature_weights, 
                                        proximity_weight=1.5, diversity_weight=1.0) # proximity_weight changed from default 0.5 to 1.5

Diverse Counterfactuals found! total time taken: 00 min 08 sec


In [16]:
dice_exp.visualize_as_dataframe()

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,22.0,Private,HS-grad,Single,Service,White,Female,45.0,0.009978



Diverse Counterfactual set (new outcome : 1)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,23.0,Private,Doctorate,Married,Professional,White,Female,74.0,0.765
1,26.0,Private,Masters,Married,White-Collar,White,Male,45.0,0.817
2,23.0,Private,Doctorate,Married,Service,White,Male,61.0,0.723
3,68.0,Private,Doctorate,Married,White-Collar,White,Female,25.0,0.816


As we see from above table, both continuous and categorical features are more closer to the original query instance and the counterfactuals are also less diverse than before.

### 4. Selecting the features to vary

While counterfactuals provide *actionable* alternative profiles to achieve a different outcome, we note that some of the generated explanations suggest changes in sensitive attributes like race or gender. Hence, DiCE allows feeding in a list of features that are allowed to vary through a *features_to_vary* parameter. 

In [17]:
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=4, desired_class="opposite", 
                                        feature_weights=feature_weights, 
                                        features_to_vary=['age','workclass','education','occupation','hours_per_week'])

Diverse Counterfactuals found! total time taken: 00 min 10 sec


In [18]:
dice_exp.visualize_as_dataframe()

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,22.0,Private,HS-grad,Single,Service,White,Female,45.0,0.009978



Diverse Counterfactual set (new outcome : 1)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,90.0,Government,Masters,Single,White-Collar,White,Female,99.0,0.717
1,90.0,Self-Employed,Doctorate,Single,White-Collar,White,Female,99.0,0.730
2,90.0,Government,Doctorate,Single,Professional,White,Female,99.0,0.732
3,90.0,Private,Masters,Single,Professional,White,Female,99.0,0.657


As we see from above table, without changing race or gender, the black-box ML model requires the continuous features to be at their maximums despite the feeding of *feature_weights* parameter. Further, the resulting counterfactuals are also not very diverse compared to previous resutls. Thus, restraining the change in *race* and *gender* exposes the edge-cases and biases in the ML model (or the data). We discuss the implications of such results in terms of model debugging and fairness evaluation in our [paper](https://arxiv.org/pdf/1905.07697.pdf) in detail. 